In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
df=pd.read_csv('resultado_consolidado.csv')
n_perguntas = len(df)/5
n = len(df)/30
print(n_perguntas, n)

1950.0 325.0


In [3]:
perguntas = np.zeros((int(n), int(n_perguntas)))
df["score"]= (df["gabarito"]==df["rotulo"])
df["score"] = df["score"].astype(int)
df['aceita']=0
print(perguntas)
df.head()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,usuario,pergunta,resposta,tipo,gabarito,rotulo,score,aceita
0,0,0,Descobrir que tu tem q tomar remédio p vida in...,2,0,0,1,0
1,0,0,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1,1,1,1,0
2,0,0,"@chimekarlla gosto muito de fazenda , mas mor...",0,1,1,1,0
3,0,0,"@behin_d_curtain Para mim, é precisamente o co...",0,0,1,0,0
4,0,0,tudo que eu não queria era estar em casa sozin...,0,0,0,1,0


In [4]:
grouped = df.groupby(['usuario', 'pergunta', 'tipo'])
for name, group in grouped:
    if (name[2] == 1 or name[2]== 2):
        if (len(group)==group['score'].sum()):
            perguntas[name[0],name[1]] += 0.5
            if (perguntas[name[0],name[1]]==1):
                df.loc[(df['usuario'] ==name[0]) & (df['pergunta'] == name[1]), 'aceita'] = 1
                
        #print (name)
        #print (group)


In [5]:
a=df[df['tipo']==0].groupby(['pergunta']).agg({'score':'count', 'aceita': 'sum'}).reset_index().rename(columns={'pergunta':"estrategia",'score':'Sentenças a rotular'})

In [6]:
#quantidade de rotulações aceitas por estratégia
a['% Aceitação']=a['aceita']*100/a['Sentenças a rotular']
a

,estrategia,Sentenças a rotular,aceita,% Aceitação
0,0,975,921,94.461538
1,1,650,600,92.307692
2,2,325,134,41.230769
3,3,650,300,46.153846
4,4,325,219,67.384615
5,5,325,251,77.230769


In [7]:
#df.groupby(['pergunta']).filter(lambda x: (x['tipo'] == 0).any()).agg({'score':'sum', 'rotulo': 'count'}).rename(columns={'score':'Total sentenças'})
#acurácia das rotulações por estratégia
b=df[(df['tipo']==0) & (df['aceita']==1)].groupby(['pergunta']).agg({'rotulo': 'count','score':'sum'}).reset_index().rename(columns={'pergunta':"estrategia",'rotulo':'Total Aceitas','score':'Acertos'})
b['Acurácia (%)'] = b['Acertos']*100/b['Total Aceitas']
b

,estrategia,Total Aceitas,Acertos,Acurácia (%)
0,0,921,503,54.614549
1,1,600,487,81.166667
2,2,134,105,78.358209
3,3,300,178,59.333333
4,4,219,168,76.712329
5,5,251,202,80.478088


In [8]:
aux1=df[df['tipo']==0].groupby(['resposta']).agg({'tipo': 'count','rotulo': 'mean','score':'sum', 'gabarito':'mean'}).reset_index().rename(columns={'tipo':'Total Respostas','rotulo':'Média','score':'Acertos'})
aux1# Média sem filtrar

,resposta,Total Respostas,Média,Acertos,gabarito
0,"@Goncla_Silva és um gajo bacano, vê lá se não ...",325,0.575385,187,1
1,@KingJokerLeto mas amiga eu to aqui ainda,325,0.246154,245,0
2,"@RivasJairo Bravo, Jairo!!!",325,0.689231,224,1
3,"@behin_d_curtain Para mim, é precisamente o co...",325,0.673846,106,0
4,"@chimekarlla gosto muito de fazenda , mas mor...",325,0.335385,109,1
5,Amanha vou ver venom :D,325,0.689231,224,1
6,Já com saudades do meu nenê,325,0.387692,199,0
7,Valoriza o coração do menininho que vc tem. El...,325,0.827692,269,1
8,tudo que eu não queria era estar em casa sozin...,325,0.015385,320,0
9,é tão lindo que dói,325,0.809231,263,1


In [10]:
aux=df[(df['tipo']==0) & (df['aceita']==1)].groupby(['resposta']).agg({'tipo': 'count','rotulo': 'mean','score':'sum', 'gabarito':'mean'}).reset_index().rename(columns={'resposta':'Sentença','tipo':'Total Respostas','rotulo':'Média','score':'Acertos'})
aux# Filtro

,Sentença,Total Respostas,Média,Acertos,gabarito
0,"@Goncla_Silva és um gajo bacano, vê lá se não ...",150,0.593333,89,1
1,@KingJokerLeto mas amiga eu to aqui ainda,300,0.233333,230,0
2,"@RivasJairo Bravo, Jairo!!!",219,0.767123,168,1
3,"@behin_d_curtain Para mim, é precisamente o co...",307,0.687296,96,0
4,"@chimekarlla gosto muito de fazenda , mas mor...",307,0.332248,102,1
5,Amanha vou ver venom :D,251,0.804781,202,1
6,Já com saudades do meu nenê,150,0.406667,89,0
7,Valoriza o coração do menininho que vc tem. El...,300,0.856667,257,1
8,tudo que eu não queria era estar em casa sozin...,307,0.006515,305,0
9,é tão lindo que dói,134,0.783582,105,1


In [11]:
aux['u <=']=aux['Média']+1.96*aux['Média']*(1-aux['Média'])/ aux['Total Respostas'].apply(lambda x: sqrt(x))
aux['u >=']=aux['Média']-1.96*aux['Média']*(1-aux['Média'])/ aux['Total Respostas'].apply(lambda x: sqrt(x))


In [12]:
#estimador do u com 95% de confiança
aux

,Sentença,Total Respostas,Média,Acertos,gabarito,u <=,u >=
0,"@Goncla_Silva és um gajo bacano, vê lá se não ...",150,0.593333,89,1,0.631948,0.554719
1,@KingJokerLeto mas amiga eu to aqui ainda,300,0.233333,230,0,0.253577,0.213090
2,"@RivasJairo Bravo, Jairo!!!",219,0.767123,168,1,0.790784,0.743463
3,"@behin_d_curtain Para mim, é precisamente o co...",307,0.687296,96,0,0.711338,0.663255
4,"@chimekarlla gosto muito de fazenda , mas mor...",307,0.332248,102,1,0.357065,0.307430
5,Amanha vou ver venom :D,251,0.804781,202,1,0.824217,0.785344
6,Já com saudades do meu nenê,150,0.406667,89,0,0.445281,0.368052
7,Valoriza o coração do menininho que vc tem. El...,300,0.856667,257,1,0.870562,0.842772
8,tudo que eu não queria era estar em casa sozin...,307,0.006515,305,0,0.007239,0.005791
9,é tão lindo que dói,134,0.783582,105,1,0.812295,0.754869
